# GameZone

### Console Wars

### Author: Wenyi Xu

Discuss the epic console war amongst **Sony**, **Microsoft** and **Nintendo**.

Scape to get some datasets for analysis and visualization to show the sales and other statistics for different platforms.

## Backgroud Information

### Playstation Release timeline

* PS1 Release Date	Saturday - December 3, 1994

* PS2 Release Date	Saturday - March 4, 2000

* PS3 Release Date	Saturday - November 11, 2006

* PS4 Release Date	Friday - Nov 15, 2013 

* PS4 Pro Release	Thursday - Nov 10, 2016 

### Playstation Release timeline

* Xbox Release Date	- November 15, 2001

* Xbox 360 Release Date	- November 22, 2005

* Xbox One Release Date	- November 22, 2013

* Xbox One X Release Date - June 13, 2016

### Nintendo Platforms release timeline

* NES Release Date - July 15, 1983

* Nintendo 64 Release Date - June 23, 1996

* GameCube Release Date - September 14, 2001

* Wii Release Date - November 19, 2006

* Wii u Release Date - November 18, 2012

* Switch Release Date - March 3, 2017


## Part 0: Scrape VGChartz

Scrape VGChartz to get useful datasets about game and hardware sales for different platforms.

I can only find sales data of home video games consoles on VGChartz starting from 25th Nov 2006 (PS3 & XBOX360 & Wii), so the data of PS2 and other previous consoles need to be extracted from other sources.

In [31]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

In [178]:
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date
from datetime import datetime
from datetime import timedelta
import math
import time

The VGChartz important pages that contain data we need

In [32]:
# Main page
vgchartz_main_url = 'http://www.vgchartz.com/'
# Gloabl weekly sales
# http://www.vgchartz.com/weekly/[date_id]/Global/
# date_id: start from 39047 (25th Nov 2006) to 43058 (18th Nov 2017)
date_id_start = 39047
date_start = datetime(2006, 11, 25)
date_id_end = 43058
date_end = datetime(2017, 11, 18)

date_id = 39047

vgchartz_weekly_url_head = 'http://www.vgchartz.com/weekly/'
vgchartz_weekly_url_tail = '/Global/'
vgchartz_weekly_url = vgchartz_weekly_url_head + str(date_id) + vgchartz_weekly_url_tail

Check if the date diff correspond to the date_id diff

In [27]:
print(datetime(2017, 11, 18) - date_start)
print(43058 - 39047)

4011 days, 0:00:00
4011


### Function get_weekly_url

**Date --> weekly_url**

We have

* Start date: date_start (25th Nov 2006)

* Start date id: date_id_start (39047)

* End date: date_end (18th Nov 2017)

* End date id: date_id_end (43058)

When given a random date between 25th Nov 2006 & 18th Nov 2017, we want to first get its corresponding week_id, and then generate its vgchartz_weekly_url


In [47]:
# date: in datetime format
def get_weekly_url(date):
    
    if type(date) != type(date_end):
        return 'Date must be type datetime'
    if date < date_start or date > date_end:
        return 'Date must be from 25th Nov 2006 to 18th Nov 2017'
    
    date_diff = date - date_start
    day_diff = date_diff.days
    # Round up
    week_diff = math.ceil(day_diff/7.0)
    date_id = date_id_start + week_diff*7
    
    vgchartz_weekly_url = vgchartz_weekly_url_head + str(date_id) + vgchartz_weekly_url_tail
    return vgchartz_weekly_url

In [52]:
# Test the function get_weekly_url
print(get_weekly_url('25th Nov 2006') == 'Date must be type datetime')
print(get_weekly_url(datetime(2000, 11, 20)) == 'Date must be from 25th Nov 2006 to 18th Nov 2017')
print(get_weekly_url(datetime(2017, 11, 20)) == 'Date must be from 25th Nov 2006 to 18th Nov 2017')
print(get_weekly_url(datetime(2007, 2, 17)) == 'http://www.vgchartz.com/weekly/39131/Global/')

True
True
True
True


Create the PoolManager

In [12]:
http = urllib3.PoolManager()

Make the request 

and query the pages

In [65]:
vgchartz_main_response = http.request('GET', vgchartz_main_url)
vgchartz_main_soup = BeautifulSoup(response.data, "lxml")

vgchartz_week_response = http.request('GET', vgchartz_weekly_url)
vgchartz_week_soup = BeautifulSoup(vgchartz_week_response.data, "lxml")

Now we have **soup**: the **HTML** of the VGCHartz pages.

Now try to extract some useful data.

There are 2 useful tables in weekly sales pages: **Global Hardware by Platform** & **Global Software by Platform**

Example of the **Global Hardware by Platform** table:

|Platform |	Weekly (change) |	Total |
|---------|-----------------|--------|
|DS	| 905,597	(+88%) |	29,319,098 |
|Wii |	529,658	(N/A) |	529,658 |
|X360 |	361,561	(+87%) |	5,943,800 |
|PSP |	352,884	(+87%) |	17,124,390 |
|PS3 |	103,130	(-50%) |	394,937 |


In [113]:
tags = vgchartz_week_soup.find_all('h2', attrs={'class': 'heading'})
tags

[<h2 class="heading">Global Hardware by Platform</h2>,
 <h2 class="heading">Global Software by Platform</h2>]

Extract the table names from the corresponding tags

In [112]:
table_names = [tag.text for tag in tags]   
table_names

['Global Hardware by Platform', 'Global Software by Platform']

### Extract the Global Hardware by Platform table

In [123]:
charts = vgchartz_week_soup.find_all('table', class_="chart")
len(charts)

4

After manually check the HTML, I found there are totally 4 charts per page.

And the 'Global Hardware by Platform' is the 3rd one -_-

In [126]:
hardware_sale_table = charts[2]
hardware_sale_table.find_all('th')

[<th align="left">Platform</th>,
 <th align="right" colspan="2">Weekly <span style="font-size:70%;">(change)</span></th>,
 <th align="right">Total</th>,
 <th align="left">Total</th>,
 <th align="right" style="padding-right:1px;">2,252,830</th>,
 <th align="left" style="padding-left:2px;"><span style="font-size:70%;">(+111%)</span>
 </th>,
 <th align="left"> </th>]

Get all the **colume** names

In [128]:
hardware_sale_cols = [x.text for x in hardware_sale_table.find_all('th')][:3]
hardware_sale_cols

['Platform', 'Weekly (change)', 'Total']

In [137]:
hardware_sale_table.find_all('td', attrs={})

[<td align="left">DS</td>,
 <td align="right" style="padding-right:1px;">905,597</td>,
 <td align="left" style="padding-left:2px;"><span style="font-size:70%;">(+88%)</span></td>,
 <td align="right">29,319,098</td>,
 <td align="left">Wii</td>,
 <td align="right" style="padding-right:1px;">529,658</td>,
 <td align="left" style="padding-left:2px;"><span style="font-size:70%;">(N/A)</span></td>,
 <td align="right">529,658</td>,
 <td align="left">X360</td>,
 <td align="right" style="padding-right:1px;">361,561</td>,
 <td align="left" style="padding-left:2px;"><span style="font-size:70%;">(+87%)</span></td>,
 <td align="right">5,943,800</td>,
 <td align="left">PSP</td>,
 <td align="right" style="padding-right:1px;">352,884</td>,
 <td align="left" style="padding-left:2px;"><span style="font-size:70%;">(+87%)</span></td>,
 <td align="right">17,124,390</td>,
 <td align="left">PS3</td>,
 <td align="right" style="padding-right:1px;">103,130</td>,
 <td align="left" style="padding-left:2px;"><span

In [140]:
hardware_sale_rows = hardware_sale_table.find_all('td', attrs={})
hardware_sale_rows = [row.text.strip() for row in hardware_sale_rows]
hardware_sale_rows

['DS',
 '905,597',
 '(+88%)',
 '29,319,098',
 'Wii',
 '529,658',
 '(N/A)',
 '529,658',
 'X360',
 '361,561',
 '(+87%)',
 '5,943,800',
 'PSP',
 '352,884',
 '(+87%)',
 '17,124,390',
 'PS3',
 '103,130',
 '(-50%)',
 '394,937']

### Function get_weekly_hardware_sales

**Date --> weekly hardware sales data**

Given a date from 25th Nov 2006 to 18th Nov 2017, we want to get which week this date is in, and get its corresponding weekly hardware sales data

In [149]:
# date: in datetime format
def get_weekly_hardware_sales(date):
    
    # Get the corresponding weekly_url from input date
    weekly_url = get_weekly_url(date)
    # Create the pool manager
    pool_manager = urllib3.PoolManager()
    
    weekly_response = pool_manager.request('GET', weekly_url)
    weekly_soup = BeautifulSoup(weekly_response.data, "lxml")
    
    # Get the weekly hardware sales table
    weekly_charts = weekly_soup.find_all('table', class_="chart")
    weekly_hardware_sale_table = weekly_charts[2]
    # Get the colume names
    # Columes should be 'Platforms', 'Weekly Sales', 'Weekly Sales Changes', 'Total Sales'
    platforms = []
    weekly_sales = []
    weekly_sales_changes = []
    total_sales = []
    weekly_hardware_sale_cols = [x.text for x in weekly_hardware_sale_table.find_all('th')][:3]
    # Get the rows
    weekly_hardware_sale_rows = weekly_hardware_sale_table.find_all('td', attrs={})
    weekly_hardware_sale_rows = [row.text.strip() for row in weekly_hardware_sale_rows]   
    # Distribute data of diff fields into diff lists
    for i in range(len(weekly_hardware_sale_rows)):
        if i%4 == 0:
            platforms.append(weekly_hardware_sale_rows[i])
        elif i%4 == 1:
            weekly_sales.append(weekly_hardware_sale_rows[i])
        elif i%4 == 2:
            weekly_sales_changes.append(weekly_hardware_sale_rows[i])
        else:
            total_sales.append(weekly_hardware_sale_rows[i])
            
    weekly_hardware_sales_df = pd.DataFrame({'Platforms': platforms, \
                                      'Weekly Sales': weekly_sales, \
                                      'Weekly Sales Changes': weekly_sales_changes, \
                                      'Total Sales': total_sales
                                     })
    return weekly_hardware_sales_df

In [150]:
date = datetime(2007, 11, 20)
get_weekly_hardware_sales(date)

,Platforms,Total Sales,Weekly Sales,Weekly Sales Changes
0,DS,"56,338,561","1,353,850",(+105%)
1,Wii,"15,192,774","673,069",(+46%)
2,PSP,"29,233,466","446,202",(+60%)
3,X360,"13,704,598","414,403",(+82%)
4,PS3,"6,638,792","376,441",(+46%)


## Part 1: Analyze the Weekly Sales

#### Wii Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [184]:
start_date = datetime(2006, 11, 25)
end_date = datetime(2017, 11, 18)

date_p = start_date
wii_sales_date = []
wii_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if Wii is in sale
    if 'Wii' in weekly_hardware_sales['Platforms'].values:
        wii_sales_date.append(date_p)
        wii_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='Wii']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.5)

In [186]:
wii_sales_df = pd.DataFrame({'Date': wii_sales_date, \
                             'Total Sales': wii_sales \
                                     })
wii_sales_df[-10:]

,Date,Total Sales
466,2015-10-31,"100,998,916"
467,2015-11-07,"100,999,541"
468,2015-11-14,"101,000,343"
469,2015-11-21,"101,001,662"
470,2015-11-28,"101,003,840"
471,2015-12-05,"101,005,477"
472,2015-12-12,"101,007,242"
473,2015-12-19,"101,009,240"
474,2015-12-26,"101,011,107"
475,2016-01-02,"101,011,951"


In [221]:
wii_sales_df.columns = ['Date', 'Wii Total Sales']
wii_sales_df[-10:]

,Date,Wii Total Sales
466,2015-10-31,"100,998,916"
467,2015-11-07,"100,999,541"
468,2015-11-14,"101,000,343"
469,2015-11-21,"101,001,662"
470,2015-11-28,"101,003,840"
471,2015-12-05,"101,005,477"
472,2015-12-12,"101,007,242"
473,2015-12-19,"101,009,240"
474,2015-12-26,"101,011,107"
475,2016-01-02,"101,011,951"


In [199]:
wii_sales_df.to_csv('./datasets/wii_sales.csv', sep='\t')

#### Wii U Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [200]:
start_date = datetime(2012, 11, 17)
end_date = datetime(2017, 11, 18)

date_p = start_date
wiiu_sales_date = []
wiiu_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if WiiU is in sale
    if 'WiiU' in weekly_hardware_sales['Platforms'].values:
        wiiu_sales_date.append(date_p)
        wiiu_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='WiiU']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.1)

In [203]:
wiiu_sales_df = pd.DataFrame({'Date': wiiu_sales_date, \
                             'Total Sales': wiiu_sales \
                                     })
wiiu_sales_df[-10:]

,Date,Total Sales
251,2017-09-16,"13,524,328"
252,2017-09-23,"13,525,073"
253,2017-09-30,"13,525,810"
254,2017-10-07,"13,526,483"
255,2017-10-14,"13,527,170"
256,2017-10-21,"13,527,821"
257,2017-10-28,"13,528,454"
258,2017-11-04,"13,529,045"
259,2017-11-11,"13,529,611"
260,2017-11-18,"13,530,264"


In [222]:
wiiu_sales_df.columns = ['Date', 'WiiU Total Sales']
wiiu_sales_df[-10:]

,Date,WiiU Total Sales
251,2017-09-16,"13,524,328"
252,2017-09-23,"13,525,073"
253,2017-09-30,"13,525,810"
254,2017-10-07,"13,526,483"
255,2017-10-14,"13,527,170"
256,2017-10-21,"13,527,821"
257,2017-10-28,"13,528,454"
258,2017-11-04,"13,529,045"
259,2017-11-11,"13,529,611"
260,2017-11-18,"13,530,264"


In [204]:
wiiu_sales_df.to_csv('./datasets/wiiu_sales.csv', sep='\t')

#### PS3 Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [187]:
start_date = datetime(2006, 11, 25)
end_date = datetime(2017, 11, 18)

date_p = start_date
ps3_sales_date = []
ps3_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if Wii is in sale
    if 'PS3' in weekly_hardware_sales['Platforms'].values:
        ps3_sales_date.append(date_p)
        ps3_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='PS3']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.1)

In [188]:
PS3_sales_df = pd.DataFrame({'Date': ps3_sales_date, \
                             'Total Sales': ps3_sales \
                                     })
PS3_sales_df[-10:]

,Date,Total Sales
564,2017-09-16,"86,892,414"
565,2017-09-23,"86,893,083"
566,2017-09-30,"86,893,656"
567,2017-10-07,"86,894,195"
568,2017-10-14,"86,894,717"
569,2017-10-21,"86,895,166"
570,2017-10-28,"86,895,649"
571,2017-11-04,"86,896,170"
572,2017-11-11,"86,896,630"
573,2017-11-18,"86,897,152"


In [223]:
PS3_sales_df.columns = ['Date', 'PS3 Total Sales']
PS3_sales_df[-10:]

,Date,PS3 Total Sales
564,2017-09-16,"86,892,414"
565,2017-09-23,"86,893,083"
566,2017-09-30,"86,893,656"
567,2017-10-07,"86,894,195"
568,2017-10-14,"86,894,717"
569,2017-10-21,"86,895,166"
570,2017-10-28,"86,895,649"
571,2017-11-04,"86,896,170"
572,2017-11-11,"86,896,630"
573,2017-11-18,"86,897,152"


In [206]:
PS3_sales_df.to_csv('./datasets/ps3_sales.csv', sep='\t')

#### XBOX 360 Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [189]:
start_date = datetime(2006, 11, 25)
end_date = datetime(2017, 11, 18)

date_p = start_date
x360_sales_date = []
x360_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if Wii is in sale
    if 'X360' in weekly_hardware_sales['Platforms'].values:
        x360_sales_date.append(date_p)
        x360_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='X360']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.1)

In [190]:
x360_sales_df = pd.DataFrame({'Date': x360_sales_date, \
                             'Total Sales': x360_sales \
                                     })
x360_sales_df[-10:]

,Date,Total Sales
564,2017-09-16,"85,807,263"
565,2017-09-23,"85,807,742"
566,2017-09-30,"85,808,186"
567,2017-10-07,"85,808,639"
568,2017-10-14,"85,809,123"
569,2017-10-21,"85,809,603"
570,2017-10-28,"85,810,055"
571,2017-11-04,"85,810,486"
572,2017-11-11,"85,810,862"
573,2017-11-18,"85,811,280"


In [224]:
x360_sales_df.columns = ['Date', 'X360 Total Sales']
x360_sales_df[-10:]

,Date,X360 Total Sales
564,2017-09-16,"85,807,263"
565,2017-09-23,"85,807,742"
566,2017-09-30,"85,808,186"
567,2017-10-07,"85,808,639"
568,2017-10-14,"85,809,123"
569,2017-10-21,"85,809,603"
570,2017-10-28,"85,810,055"
571,2017-11-04,"85,810,486"
572,2017-11-11,"85,810,862"
573,2017-11-18,"85,811,280"


In [207]:
x360_sales_df.to_csv('./datasets/x360_sales.csv', sep='\t')

#### Nintendo Switch Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [268]:
start_date = datetime(2013, 8, 17)
end_date = datetime(2017, 11, 18)

date_p = start_date
ns_sales_date = []
ns_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if Wii is in sale
    if 'NS' in weekly_hardware_sales['Platforms'].values:
        ns_sales_date.append(date_p)
        ns_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='NS']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.01)

In [269]:
ns_sales_df = pd.DataFrame({'Date': ns_sales_date, \
                             'Total Sales': ns_sales \
                                     })
ns_sales_df[-10:]

,Date,Total Sales
28,2017-09-16,"5,925,303"
29,2017-09-23,"6,081,881"
30,2017-09-30,"6,282,240"
31,2017-10-07,"6,437,927"
32,2017-10-14,"6,605,881"
33,2017-10-21,"6,770,056"
34,2017-10-28,"7,094,564"
35,2017-11-04,"7,329,519"
36,2017-11-11,"7,606,357"
37,2017-11-18,"7,994,004"


In [270]:
ns_sales_df.columns = ['Date', 'NS Total Sales']
ns_sales_df[-10:]

,Date,NS Total Sales
28,2017-09-16,"5,925,303"
29,2017-09-23,"6,081,881"
30,2017-09-30,"6,282,240"
31,2017-10-07,"6,437,927"
32,2017-10-14,"6,605,881"
33,2017-10-21,"6,770,056"
34,2017-10-28,"7,094,564"
35,2017-11-04,"7,329,519"
36,2017-11-11,"7,606,357"
37,2017-11-18,"7,994,004"


In [271]:
ns_sales_df.to_csv('./datasets/ns_sales.csv', sep='\t')

#### PS4 Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [193]:
start_date = datetime(2013, 6, 1)
end_date = datetime(2017, 11, 18)

date_p = start_date
ps4_sales_date = []
ps4_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if PS4 is in sale
    if 'PS4' in weekly_hardware_sales['Platforms'].values:
        ps4_sales_date.append(date_p)
        ps4_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='PS4']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.1)

In [195]:
ps4_sales_df = pd.DataFrame({'Date': ps4_sales_date, \
                             'Total Sales': ps4_sales \
                                     })
ps4_sales_df[-10:]

,Date,Total Sales
200,2017-09-16,"64,352,281"
201,2017-09-23,"64,609,497"
202,2017-09-30,"64,909,187"
203,2017-10-07,"65,177,958"
204,2017-10-14,"65,442,943"
205,2017-10-21,"65,750,221"
206,2017-10-28,"66,031,358"
207,2017-11-04,"66,490,554"
208,2017-11-11,"67,170,279"
209,2017-11-18,"67,977,292"


In [226]:
ps4_sales_df.columns = ['Date', 'PS4 Total Sales']
ps4_sales_df[-10:]

,Date,PS4 Total Sales
200,2017-09-16,"64,352,281"
201,2017-09-23,"64,609,497"
202,2017-09-30,"64,909,187"
203,2017-10-07,"65,177,958"
204,2017-10-14,"65,442,943"
205,2017-10-21,"65,750,221"
206,2017-10-28,"66,031,358"
207,2017-11-04,"66,490,554"
208,2017-11-11,"67,170,279"
209,2017-11-18,"67,977,292"


In [209]:
ps4_sales_df.to_csv('./datasets/ps4_sales.csv', sep='\t')

#### XBOX ONE Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [196]:
start_date = datetime(2013, 8, 17)
end_date = datetime(2017, 11, 18)

date_p = start_date
xone_sales_date = []
xone_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if PS4 is in sale
    if 'XOne' in weekly_hardware_sales['Platforms'].values:
        xone_sales_date.append(date_p)
        xone_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='XOne']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.1)

In [265]:
xone_sales_df = pd.DataFrame({'Date': xone_sales_date, \
                             'Total Sales': xone_sales \
                                     })
xone_sales_df[-10:]

,Date,Total Sales
199,2017-09-16,"30,989,732"
200,2017-09-23,"31,070,354"
201,2017-09-30,"31,176,640"
202,2017-10-07,"31,270,019"
203,2017-10-14,"31,364,608"
204,2017-10-21,"31,464,018"
205,2017-10-28,"31,573,460"
206,2017-11-04,"31,733,475"
207,2017-11-11,"32,562,278"
208,2017-11-18,"33,017,552"


In [279]:
xone_sales_df.columns = ['Date', 'XONE Total Sales']
xone_sales_df[-10:]

,Date,XONE Total Sales
199,2017-09-16,"30,989,732"
200,2017-09-23,"31,070,354"
201,2017-09-30,"31,176,640"
202,2017-10-07,"31,270,019"
203,2017-10-14,"31,364,608"
204,2017-10-21,"31,464,018"
205,2017-10-28,"31,573,460"
206,2017-11-04,"31,733,475"
207,2017-11-11,"32,562,278"
208,2017-11-18,"33,017,552"


In [201]:
xone_sales_df.to_csv('./datasets/xone_sales.csv', sep='\t')

List console sales together

In [280]:
sales_df = wii_sales_df.join(wiiu_sales_df.set_index('Date'), on='Date', how='outer')\
            .join(ns_sales_df.set_index('Date'), on='Date', how='outer')\
            .join(PS3_sales_df.set_index('Date'), on='Date', how='outer')\
            .join(ps4_sales_df.set_index('Date'), on='Date', how='outer')\
            .join(x360_sales_df.set_index('Date'), on='Date', how='outer')\
            .join(xone_sales_df.set_index('Date'), on='Date', how='outer')
sales_df = sales_df.fillna('0')
sales_df

,Date,Wii Total Sales,WiiU Total Sales,NS Total Sales,PS3 Total Sales,PS4 Total Sales,X360 Total Sales,XONE Total Sales
0,2006-11-25,"529,658",0,0,"394,937",0,"5,943,800",0
1,2006-12-02,"1,067,537",0,0,"524,687",0,"6,264,980",0
2,2006-12-09,"1,656,510",0,0,"687,424",0,"6,711,905",0
3,2006-12-16,"2,071,242",0,0,"887,425",0,"7,212,617",0
4,2006-12-23,"2,669,210",0,0,"1,078,020",0,"7,783,860",0
5,2006-12-30,"2,954,781",0,0,"1,252,040",0,"7,990,455",0
6,2007-01-06,"3,258,928",0,0,"1,388,651",0,"8,138,246",0
7,2007-01-13,"3,510,426",0,0,"1,478,133",0,"8,261,585",0
8,2007-01-20,"3,755,861",0,0,"1,557,916",0,"8,377,993",0
9,2007-01-27,"4,007,149",0,0,"1,632,136",0,"8,491,072",0


In [283]:
sales_df['Wii Total Sales'] = sales_df['Wii Total Sales'].apply(lambda x: int(str(x).replace(',', '')))
sales_df['WiiU Total Sales'] = sales_df['WiiU Total Sales'].apply(lambda x: int(str(x).replace(',', '')))
sales_df['NS Total Sales'] = sales_df['NS Total Sales'].apply(lambda x: int(str(x).replace(',', '')))
sales_df['PS3 Total Sales'] = sales_df['PS3 Total Sales'].apply(lambda x: int(str(x).replace(',', '')))
sales_df['PS4 Total Sales'] = sales_df['PS4 Total Sales'].apply(lambda x: int(str(x).replace(',', '')))
sales_df['X360 Total Sales'] = sales_df['X360 Total Sales'].apply(lambda x: int(str(x).replace(',', '')))
sales_df['XONE Total Sales'] = sales_df['XONE Total Sales'].apply(lambda x: int(str(x).replace(',', '')))
sales_df['Total Sales'] = sales_df['Wii Total Sales'] + sales_df['WiiU Total Sales'] +\
                          sales_df['PS3 Total Sales'] + sales_df['PS4 Total Sales'] +\
                          sales_df['X360 Total Sales'] + sales_df['XONE Total Sales']
sales_df['Nintendo Total Sales'] = sales_df['Wii Total Sales'] + sales_df['WiiU Total Sales'] + 
sales_df['Sony Total Sales'] = sales_df['PS3 Total Sales'] + sales_df['PS4 Total Sales']
sales_df['Microsoft Total Sales'] = sales_df['X360 Total Sales'] + sales_df['XONE Total Sales']

sales_df

,Date,Wii Total Sales,WiiU Total Sales,NS Total Sales,PS3 Total Sales,PS4 Total Sales,X360 Total Sales,XONE Total Sales,Total Sales,Nintendo Total Sales,Sony Total Sales,Microsoft Total Sales
0,2006-11-25,529658,0,0,394937,0,5943800,0,6868395,529658,394937,5943800
1,2006-12-02,1067537,0,0,524687,0,6264980,0,7857204,1067537,524687,6264980
2,2006-12-09,1656510,0,0,687424,0,6711905,0,9055839,1656510,687424,6711905
3,2006-12-16,2071242,0,0,887425,0,7212617,0,10171284,2071242,887425,7212617
4,2006-12-23,2669210,0,0,1078020,0,7783860,0,11531090,2669210,1078020,7783860
5,2006-12-30,2954781,0,0,1252040,0,7990455,0,12197276,2954781,1252040,7990455
6,2007-01-06,3258928,0,0,1388651,0,8138246,0,12785825,3258928,1388651,8138246
7,2007-01-13,3510426,0,0,1478133,0,8261585,0,13250144,3510426,1478133,8261585
8,2007-01-20,3755861,0,0,1557916,0,8377993,0,13691770,3755861,1557916,8377993
9,2007-01-27,4007149,0,0,1632136,0,8491072,0,14130357,4007149,1632136,8491072


In [287]:
sales_df['Nintendo Share'] = sales_df['Nintendo Total Sales'] / sales_df['Total Sales']
sales_df['Sony Share'] = sales_df['Sony Total Sales'] / sales_df['Total Sales']
sales_df['Microsoft Share'] = sales_df['Microsoft Total Sales'] / sales_df['Total Sales']
sales_df

,Date,Wii Total Sales,WiiU Total Sales,NS Total Sales,PS3 Total Sales,PS4 Total Sales,X360 Total Sales,XONE Total Sales,Total Sales,Nintendo Total Sales,Sony Total Sales,Microsoft Total Sales,Nintendo Share,Sony Share,Microsoft Share
0,2006-11-25,529658,0,0,394937,0,5943800,0,6868395,529658,394937,5943800,0.077115,0.057501,0.865384
1,2006-12-02,1067537,0,0,524687,0,6264980,0,7857204,1067537,524687,6264980,0.135867,0.066778,0.797355
2,2006-12-09,1656510,0,0,687424,0,6711905,0,9055839,1656510,687424,6711905,0.182922,0.075909,0.741169
3,2006-12-16,2071242,0,0,887425,0,7212617,0,10171284,2071242,887425,7212617,0.203636,0.087248,0.709116
4,2006-12-23,2669210,0,0,1078020,0,7783860,0,11531090,2669210,1078020,7783860,0.231479,0.093488,0.675032
5,2006-12-30,2954781,0,0,1252040,0,7990455,0,12197276,2954781,1252040,7990455,0.242249,0.102649,0.655102
6,2007-01-06,3258928,0,0,1388651,0,8138246,0,12785825,3258928,1388651,8138246,0.254886,0.108609,0.636505
7,2007-01-13,3510426,0,0,1478133,0,8261585,0,13250144,3510426,1478133,8261585,0.264935,0.111556,0.623509
8,2007-01-20,3755861,0,0,1557916,0,8377993,0,13691770,3755861,1557916,8377993,0.274315,0.113785,0.611900
9,2007-01-27,4007149,0,0,1632136,0,8491072,0,14130357,4007149,1632136,8491072,0.283584,0.115506,0.600910


In [298]:
sony_share_df = sales_df['Sony Share']
sony_share = sony_share_df.values.tolist()

In [302]:
microsoft_share_df = sales_df['Microsoft Share']
microsoft_share = microsoft_share_df.values.tolist()
nintendo_share_df = sales_df['Nintendo Share']
nintendo_share = nintendo_share_df.values.tolist()

In [303]:
print(len(sony_share))
print(len(microsoft_share))
print(len(nintendo_share))

574
574
574


In [301]:
sony_share_per_num = [round(x*100, 1) for x in sony_share]
print(sony_share_per_num)

[5.8, 6.7, 7.6, 8.7, 9.3, 10.3, 10.9, 11.2, 11.4, 11.6, 11.7, 11.8, 11.9, 11.9, 12.1, 12.4, 12.5, 15.2, 15.8, 16.1, 16.3, 16.4, 16.4, 16.4, 16.5, 16.5, 16.5, 16.5, 16.5, 16.5, 16.4, 16.4, 16.4, 16.5, 16.6, 16.7, 16.9, 16.9, 17.0, 17.0, 17.1, 17.1, 17.1, 17.1, 17.1, 17.1, 17.3, 17.5, 17.6, 17.8, 18.1, 18.4, 18.7, 19.0, 19.5, 20.0, 20.3, 20.6, 20.8, 21.0, 21.1, 21.2, 21.3, 21.4, 21.5, 21.6, 21.7, 21.7, 21.8, 21.8, 21.9, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.1, 22.0, 22.0, 22.0, 22.2, 22.3, 22.3, 22.3, 22.3, 22.3, 22.3, 22.3, 22.2, 22.3, 22.3, 22.4, 22.4, 22.4, 22.4, 22.4, 22.4, 22.4, 22.3, 22.3, 22.3, 22.3, 22.2, 22.1, 21.9, 21.9, 21.8, 21.7, 21.7, 21.7, 21.8, 21.8, 21.8, 21.8, 21.8, 21.8, 21.8, 21.8, 21.9, 21.9, 21.9, 21.9, 22.0, 22.0, 22.0, 22.0, 22.0, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.2, 22.2, 22.2, 22.2, 22.2, 22.2, 22.2, 22.2, 22.3, 22.7, 22.8, 23.0, 23.1, 23.2, 23.3, 23.4, 23.4, 23.5, 23.6, 23.7, 23.7, 23.9, 23.9, 24.0, 24.1, 24.2, 24.3, 24.4, 24.5, 24.5, 24.6, 24

In [304]:
microsoft_share_per_num = [round(x*100, 1) for x in microsoft_share]
print(microsoft_share_per_num)

[86.5, 79.7, 74.1, 70.9, 67.5, 65.5, 63.7, 62.4, 61.2, 60.1, 59.1, 58.3, 57.6, 56.9, 56.1, 55.5, 54.9, 52.6, 51.9, 51.2, 50.6, 49.9, 49.2, 48.5, 48.0, 47.5, 47.0, 46.5, 46.0, 45.6, 45.2, 44.7, 44.3, 43.7, 43.2, 42.6, 42.2, 41.8, 41.4, 41.2, 40.9, 40.7, 40.5, 40.5, 40.5, 40.6, 40.5, 40.2, 39.9, 39.7, 39.4, 39.0, 38.6, 38.0, 37.5, 37.1, 36.1, 35.7, 35.4, 35.3, 35.1, 34.9, 34.7, 34.5, 34.4, 34.3, 34.1, 33.9, 33.8, 33.6, 33.5, 33.3, 33.1, 32.9, 32.8, 32.5, 32.4, 32.3, 32.0, 31.9, 31.8, 31.6, 31.4, 31.2, 31.1, 30.9, 30.9, 30.8, 30.7, 30.7, 30.6, 30.5, 30.4, 30.3, 30.3, 30.3, 30.3, 30.3, 30.2, 30.2, 30.2, 30.1, 30.1, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 29.9, 29.9, 29.8, 29.8, 29.8, 29.7, 29.7, 29.7, 29.7, 29.7, 29.7, 29.6, 29.6, 29.6, 29.6, 29.6, 29.5, 29.5, 29.5, 29.5, 29.5, 29.5, 29.5, 29.5, 29.5, 29.5, 29.5, 29.5, 29.4, 29.4, 29.4, 29.4, 29.4, 29.3, 29.2, 29.2, 29.2, 29.1, 29.0, 29.0, 28.9, 28.8, 28.8, 28.9, 28.8, 28.7, 28.5, 28.3, 28.1, 27.9, 27.7, 27.7, 27.6, 27.6, 27.

In [306]:
nintendo_share_per_num = [round(x*100, 1) for x in nintendo_share]
print(nintendo_share_per_num)

[7.7, 13.6, 18.3, 20.4, 23.1, 24.2, 25.5, 26.5, 27.4, 28.4, 29.1, 29.8, 30.5, 31.2, 31.7, 32.2, 32.7, 32.2, 32.3, 32.7, 33.1, 33.7, 34.4, 35.0, 35.5, 36.0, 36.5, 37.0, 37.5, 37.9, 38.4, 38.9, 39.3, 39.8, 40.3, 40.6, 41.0, 41.3, 41.6, 41.8, 42.0, 42.2, 42.3, 42.4, 42.3, 42.3, 42.2, 42.3, 42.5, 42.5, 42.4, 42.6, 42.8, 43.0, 43.1, 42.9, 43.6, 43.8, 43.8, 43.7, 43.8, 43.9, 44.0, 44.1, 44.1, 44.2, 44.3, 44.3, 44.4, 44.6, 44.6, 44.8, 44.9, 45.1, 45.2, 45.5, 45.6, 45.7, 46.0, 46.1, 46.3, 46.2, 46.3, 46.5, 46.6, 46.7, 46.8, 47.0, 47.0, 47.1, 47.2, 47.2, 47.3, 47.3, 47.3, 47.4, 47.4, 47.4, 47.4, 47.5, 47.5, 47.6, 47.6, 47.7, 47.9, 48.1, 48.1, 48.2, 48.4, 48.3, 48.3, 48.3, 48.3, 48.3, 48.4, 48.4, 48.5, 48.5, 48.5, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.3, 48.3, 48.3, 48.4, 48.4, 48.4, 48.4, 48.4, 48.1, 47.9, 47.8, 47.7, 47.7, 47.7, 47.7, 47.7, 47.6, 47.6, 47.4, 47.4, 47.4, 47.6, 47.7, 47.8, 47.9, 47.9, 47.9, 47.9, 47.9, 47.8

In [300]:
sony_share_per = [str(round(x*100, 1))+'%' for x in sony_share]
print(sony_share_per)

['5.8%', '6.7%', '7.6%', '8.7%', '9.3%', '10.3%', '10.9%', '11.2%', '11.4%', '11.6%', '11.7%', '11.8%', '11.9%', '11.9%', '12.1%', '12.4%', '12.5%', '15.2%', '15.8%', '16.1%', '16.3%', '16.4%', '16.4%', '16.4%', '16.5%', '16.5%', '16.5%', '16.5%', '16.5%', '16.5%', '16.4%', '16.4%', '16.4%', '16.5%', '16.6%', '16.7%', '16.9%', '16.9%', '17.0%', '17.0%', '17.1%', '17.1%', '17.1%', '17.1%', '17.1%', '17.1%', '17.3%', '17.5%', '17.6%', '17.8%', '18.1%', '18.4%', '18.7%', '19.0%', '19.5%', '20.0%', '20.3%', '20.6%', '20.8%', '21.0%', '21.1%', '21.2%', '21.3%', '21.4%', '21.5%', '21.6%', '21.7%', '21.7%', '21.8%', '21.8%', '21.9%', '22.0%', '22.0%', '22.0%', '22.0%', '22.0%', '22.0%', '22.1%', '22.0%', '22.0%', '22.0%', '22.2%', '22.3%', '22.3%', '22.3%', '22.3%', '22.3%', '22.3%', '22.3%', '22.2%', '22.3%', '22.3%', '22.4%', '22.4%', '22.4%', '22.4%', '22.4%', '22.4%', '22.4%', '22.3%', '22.3%', '22.3%', '22.3%', '22.2%', '22.1%', '21.9%', '21.9%', '21.8%', '21.7%', '21.7%', '21.7%', '21.8

In [305]:
microsoft_share_per = [str(round(x*100, 1))+'%' for x in microsoft_share]
print(microsoft_share_per)

['86.5%', '79.7%', '74.1%', '70.9%', '67.5%', '65.5%', '63.7%', '62.4%', '61.2%', '60.1%', '59.1%', '58.3%', '57.6%', '56.9%', '56.1%', '55.5%', '54.9%', '52.6%', '51.9%', '51.2%', '50.6%', '49.9%', '49.2%', '48.5%', '48.0%', '47.5%', '47.0%', '46.5%', '46.0%', '45.6%', '45.2%', '44.7%', '44.3%', '43.7%', '43.2%', '42.6%', '42.2%', '41.8%', '41.4%', '41.2%', '40.9%', '40.7%', '40.5%', '40.5%', '40.5%', '40.6%', '40.5%', '40.2%', '39.9%', '39.7%', '39.4%', '39.0%', '38.6%', '38.0%', '37.5%', '37.1%', '36.1%', '35.7%', '35.4%', '35.3%', '35.1%', '34.9%', '34.7%', '34.5%', '34.4%', '34.3%', '34.1%', '33.9%', '33.8%', '33.6%', '33.5%', '33.3%', '33.1%', '32.9%', '32.8%', '32.5%', '32.4%', '32.3%', '32.0%', '31.9%', '31.8%', '31.6%', '31.4%', '31.2%', '31.1%', '30.9%', '30.9%', '30.8%', '30.7%', '30.7%', '30.6%', '30.5%', '30.4%', '30.3%', '30.3%', '30.3%', '30.3%', '30.3%', '30.2%', '30.2%', '30.2%', '30.1%', '30.1%', '30.0%', '30.0%', '30.0%', '30.0%', '30.0%', '30.0%', '30.0%', '30.0%', 

In [307]:
nintendo_share_per = [str(round(x*100, 1))+'%' for x in nintendo_share]
print(nintendo_share_per)

['7.7%', '13.6%', '18.3%', '20.4%', '23.1%', '24.2%', '25.5%', '26.5%', '27.4%', '28.4%', '29.1%', '29.8%', '30.5%', '31.2%', '31.7%', '32.2%', '32.7%', '32.2%', '32.3%', '32.7%', '33.1%', '33.7%', '34.4%', '35.0%', '35.5%', '36.0%', '36.5%', '37.0%', '37.5%', '37.9%', '38.4%', '38.9%', '39.3%', '39.8%', '40.3%', '40.6%', '41.0%', '41.3%', '41.6%', '41.8%', '42.0%', '42.2%', '42.3%', '42.4%', '42.3%', '42.3%', '42.2%', '42.3%', '42.5%', '42.5%', '42.4%', '42.6%', '42.8%', '43.0%', '43.1%', '42.9%', '43.6%', '43.8%', '43.8%', '43.7%', '43.8%', '43.9%', '44.0%', '44.1%', '44.1%', '44.2%', '44.3%', '44.3%', '44.4%', '44.6%', '44.6%', '44.8%', '44.9%', '45.1%', '45.2%', '45.5%', '45.6%', '45.7%', '46.0%', '46.1%', '46.3%', '46.2%', '46.3%', '46.5%', '46.6%', '46.7%', '46.8%', '47.0%', '47.0%', '47.1%', '47.2%', '47.2%', '47.3%', '47.3%', '47.3%', '47.4%', '47.4%', '47.4%', '47.4%', '47.5%', '47.5%', '47.6%', '47.6%', '47.7%', '47.9%', '48.1%', '48.1%', '48.2%', '48.4%', '48.3%', '48.3%', '

In [322]:
#start_date = datetime(2006, 11, 25)
#end_date = datetime(2017, 11, 18)
date_li = [datetime(2006, 11, 25) + x*timedelta(7) for x in range(574)]
date_li_day = [str(x).split()[0] for x in date_li]
print(date_li_day)

['2006-11-25', '2006-12-02', '2006-12-09', '2006-12-16', '2006-12-23', '2006-12-30', '2007-01-06', '2007-01-13', '2007-01-20', '2007-01-27', '2007-02-03', '2007-02-10', '2007-02-17', '2007-02-24', '2007-03-03', '2007-03-10', '2007-03-17', '2007-03-24', '2007-03-31', '2007-04-07', '2007-04-14', '2007-04-21', '2007-04-28', '2007-05-05', '2007-05-12', '2007-05-19', '2007-05-26', '2007-06-02', '2007-06-09', '2007-06-16', '2007-06-23', '2007-06-30', '2007-07-07', '2007-07-14', '2007-07-21', '2007-07-28', '2007-08-04', '2007-08-11', '2007-08-18', '2007-08-25', '2007-09-01', '2007-09-08', '2007-09-15', '2007-09-22', '2007-09-29', '2007-10-06', '2007-10-13', '2007-10-20', '2007-10-27', '2007-11-03', '2007-11-10', '2007-11-17', '2007-11-24', '2007-12-01', '2007-12-08', '2007-12-15', '2007-12-22', '2007-12-29', '2008-01-05', '2008-01-12', '2008-01-19', '2008-01-26', '2008-02-02', '2008-02-09', '2008-02-16', '2008-02-23', '2008-03-01', '2008-03-08', '2008-03-15', '2008-03-22', '2008-03-29', '2008